# Helpful utility functions

This notebook demonstrates how to use various utility functions related to interacting with CoLink storage. 

Every served flow has associated entries in CoLink storage. The base path to these entries corresponds to the flow_endpoint argument passed when serving the flow. The typical structure includes:
```
endpoint/
    init = 0/1
    singleton = 0/1
    parallel_dispatch = 0/1
    flow_class_name = ...
    default_dispatch_point = ...
    mounts/
       id_of_user_who_mounted_the_flow/
           flow_instance_id/
               init = 0/1
               config_overrides = ...
```

Calling serve_flow will create the initial entries under an endpoint and calling get_flow_instance will create an entry under the mounts/ path. If a Flow was served as a singleton, there will only be at most one entry under the mounts/ path.

You can see the file structure visually by connecting [this frontend](https://colink.run/) to your CoLink server.

Util functions include:
- serve_utils.is_flow_served
- serve_utils.unserve_flow
- serve_utils.delete_flow_instance
- serve_utils.delete_flow_endpoint
- serve_utils.delete_all_flow_endpoints
- colink_utils.print_flow_instances
- colink_utils.print_served_flows
- serve_utils._get_local_flow_instance_metadata

In [1]:
%load_ext autoreload
%autoreload 2
import os, json
from colink import CoLink
from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread

In [2]:
cl = colink_utils.start_colink_server()

In [3]:
run_dispatch_worker_thread(cl)
run_dispatch_worker_thread(cl)

[2024-03-23 06:45:34,343][aiflows.workers.dispatch_worker:236][INFO] - Dispatch worker started in attached thread.
[2024-03-23 06:45:34,349][aiflows.workers.dispatch_worker:237][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 06:45:34,352][aiflows.workers.dispatch_worker:236][INFO] - Dispatch worker started in attached thread.
[2024-03-23 06:45:34,353][aiflows.workers.dispatch_worker:237][INFO] - dispatch_point: coflows_dispatch


In [4]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic"
)
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential"
)

[2024-03-23 06:45:34,461][aiflows.utils.serve_utils:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
[2024-03-23 06:45:34,462][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 06:45:34,463][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-23 06:45:34,463][aiflows.utils.serve_utils:119][INFO] - singleton: False

[2024-03-23 06:45:34,488][aiflows.utils.serve_utils:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
[2024-03-23 06:45:34,489][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 06:45:34,490][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-23 06:45:34,491][aiflows.utils.serve_utils:119][INFO] - singleton: False



True

In [5]:
colink_utils.print_served_flows(cl, print_values = True)

 reverse_number_atomic/
   parallel_dispatch: False
   init: 1
   flow_class_name: ReverseNumberFlowModule.ReverseNumberAtomicFlow
   default_dispatch_point: coflows_dispatch
   singleton: False
 reverse_number_sequential/
   flow_class_name: ReverseNumberFlowModule.ReverseNumberSequentialFlow
   parallel_dispatch: False
   init: 1
   default_dispatch_point: coflows_dispatch
   singleton: False


In [6]:
flow = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_sequential",
)
# we mount 3 instances: one ReverseNumberSequentialFlow and two ReverseNumberAtomicFlow

[2024-03-23 06:45:34,813][aiflows.utils.serve_utils:336][INFO] - Mounted 70ff9eba-ae5d-4a9f-a693-4fdd72de55fc at flows:reverse_number_atomic:mounts:local:70ff9eba-ae5d-4a9f-a693-4fdd72de55fc
[2024-03-23 06:45:34,861][aiflows.utils.serve_utils:336][INFO] - Mounted 6713303c-e12e-4bdc-94fd-f0540db5fccf at flows:reverse_number_atomic:mounts:local:6713303c-e12e-4bdc-94fd-f0540db5fccf
[2024-03-23 06:45:34,879][aiflows.utils.serve_utils:336][INFO] - Mounted 2de56c70-db2d-403d-9767-bed833555673 at flows:reverse_number_sequential:mounts:local:2de56c70-db2d-403d-9767-bed833555673


In [7]:
colink_utils.print_served_flows(cl)
print("\nFlow Instances:")
colink_utils.print_flow_instances(cl)

 reverse_number_atomic/
   parallel_dispatch
   mounts/
     local/
       70ff9eba-ae5d-4a9f-a693-4fdd72de55fc/
         init
         config_overrides
       6713303c-e12e-4bdc-94fd-f0540db5fccf/
         config_overrides
         init
   init
   flow_class_name
   default_dispatch_point
   singleton
 reverse_number_sequential/
   flow_class_name
   mounts/
     local/
       2de56c70-db2d-403d-9767-bed833555673/
         init
         config_overrides
   parallel_dispatch
   init
   default_dispatch_point
   singleton

Flow Instances:
 70ff9eba-ae5d-4a9f-a693-4fdd72de55fc
 6713303c-e12e-4bdc-94fd-f0540db5fccf
 2de56c70-db2d-403d-9767-bed833555673


In [8]:
serve_utils._get_local_flow_instance_metadata(cl, flow_id=flow.get_instance_id())

{'flow_endpoint': 'reverse_number_sequential', 'user_id': 'local'}

In [9]:
serve_utils.unserve_flow(cl, flow_endpoint="reverse_number_atomic")
# just sets init entry to 0, but all entries remain in storage
# flow instances of unserved endpoint remain alive

[2024-03-23 06:45:35,470][aiflows.utils.serve_utils:199][INFO] - Stopped serving at flows:reverse_number_atomic


In [10]:
colink_utils.print_served_flows(cl)
print("\nFlow Instances:")
colink_utils.print_flow_instances(cl)

 reverse_number_atomic/
   parallel_dispatch
   init
   mounts/
     local/
       70ff9eba-ae5d-4a9f-a693-4fdd72de55fc/
         init
         config_overrides
       6713303c-e12e-4bdc-94fd-f0540db5fccf/
         config_overrides
         init
   flow_class_name
   default_dispatch_point
   singleton
 reverse_number_sequential/
   flow_class_name
   mounts/
     local/
       2de56c70-db2d-403d-9767-bed833555673/
         init
         config_overrides
   parallel_dispatch
   init
   default_dispatch_point
   singleton

Flow Instances:
 70ff9eba-ae5d-4a9f-a693-4fdd72de55fc
 6713303c-e12e-4bdc-94fd-f0540db5fccf
 2de56c70-db2d-403d-9767-bed833555673


In [11]:
serve_utils.is_flow_served(cl, flow_endpoint="reverse_number_atomic")

False

In [12]:
flow = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_sequential",
) # throws exception because a subflow is not served

FlowInstanceException: Failed to get flow instance at reverse_number_sequential served by user 03db3f52d6e40d2beccdd3c0adca0b80356cc0806e9797d2e7ebd6ec17d65eb9b9.
Message: Failed to get instance of subflow first_reverse_flow.
Failed to get flow instance at reverse_number_atomic served by user 03db3f52d6e40d2beccdd3c0adca0b80356cc0806e9797d2e7ebd6ec17d65eb9b9.
Message: Not serving at reverse_number_atomic.

In [13]:
serve_utils.delete_flow_endpoint(cl, flow_endpoint="reverse_number_atomic")
# actually deletes entries at endpoint, including all flow instances mounted on the endpoint

[2024-03-23 06:46:49,555][aiflows.utils.serve_utils:149][INFO] - Deleted flow instance 70ff9eba-ae5d-4a9f-a693-4fdd72de55fc
[2024-03-23 06:46:49,558][aiflows.utils.serve_utils:149][INFO] - Deleted flow instance 6713303c-e12e-4bdc-94fd-f0540db5fccf
[2024-03-23 06:46:49,669][aiflows.utils.serve_utils:158][INFO] - Stopped serving at flows:reverse_number_atomic


In [14]:
colink_utils.print_served_flows(cl)
print("\nFlow Instances:")
colink_utils.print_flow_instances(cl)
# note that paths under reverse_number_atomic/ are empty (even though they get printed)

 reverse_number_atomic/
   mounts/
     local/
       70ff9eba-ae5d-4a9f-a693-4fdd72de55fc/
       6713303c-e12e-4bdc-94fd-f0540db5fccf/
 reverse_number_sequential/
   flow_class_name
   mounts/
     local/
       2de56c70-db2d-403d-9767-bed833555673/
         init
         config_overrides
   parallel_dispatch
   init
   default_dispatch_point
   singleton

Flow Instances:
 2de56c70-db2d-403d-9767-bed833555673


In [15]:
serve_utils.delete_flow_instance(cl, flow_id=flow.get_instance_id())
# delets a single flow instance

[2024-03-23 06:46:54,780][aiflows.utils.serve_utils:242][INFO] - Deleted flow instance 2de56c70-db2d-403d-9767-bed833555673.


In [16]:
colink_utils.print_served_flows(cl)
print("\nFlow Instances:")
colink_utils.print_flow_instances(cl)

 reverse_number_atomic/
   mounts/
     local/
       70ff9eba-ae5d-4a9f-a693-4fdd72de55fc/
       6713303c-e12e-4bdc-94fd-f0540db5fccf/
 reverse_number_sequential/
   flow_class_name
   mounts/
     local/
       2de56c70-db2d-403d-9767-bed833555673/
   parallel_dispatch
   init
   default_dispatch_point
   singleton

Flow Instances:


In [19]:
serve_utils.delete_all_flow_endpoints(cl)
# deletes all CoLink entries associated with Flows

In [18]:
colink_utils.print_served_flows(cl)
print("\nFlow Instances:")
colink_utils.print_flow_instances(cl)
# note that paths are empty (even though they get printed)

 reverse_number_atomic/
   mounts/
     local/
       70ff9eba-ae5d-4a9f-a693-4fdd72de55fc/
       6713303c-e12e-4bdc-94fd-f0540db5fccf/
 reverse_number_sequential/
   mounts/
     local/
       2de56c70-db2d-403d-9767-bed833555673/

Flow Instances:
